In [1]:
import os
import re 
import urllib

import requests as rq

def multiple_replace(dic, text):
  # Create a regex from the dict keys
  regex = re.compile('(%s)' % '|'.join(map(re.escape, dic.keys())))

  # For each match, lookup corresponding value in dict
  return regex.sub(lambda mo: dic[mo.string[mo.start():mo.end()]], text) 

## Load latest version notebook CSS
+ Latest version is 5.1.0 - as of 21sep17 - from the jupyter notebook repo: [here](https://cdn.jupyter.org/notebook/5.1.0/style/style.min.css)
+ For some unknown reason nbconvert is stuck at notebook css version 4.3.0 - See [setup.py](https://github.com/jupyter/nbconvert/blob/8f8ac1a8cf9aa718e9a8e6c9bc986d7c052ef5d2/setup.py#L69) 

In [2]:
def load_remote_file(url, login=None, password=None,
                     proxy_host=None, proxy_port=None,
                     verbose=False):

    is_proxy = not((login is None) and (password is None) and
                   (proxy_host is None) and (proxy_port is None))
    


    if is_proxy:
        pwd = urllib.parse.quote(password)
        dic = {'login': login, 'pwd': pwd,
               'proxy_host': proxy_host, 'proxy_port': proxy_port}
        proxies = {
            'http': 'http://{login}:{pwd}@{proxy_host}:{proxy_port}'.format(**dic),
            'https': 'https://{login}:{pwd}@{proxy_host}:{proxy_port}'.format(**dic)
        }
        if verbose:
            print('pwd:', pwd)
            print('proxies:', proxies)

    res = rq.get(url, proxies=proxies, verify=False)
    assert res.status_code == 200, 'Download error'
    return res.content.decode('utf-8')

    
def write_file(string, path):
    with open(path, 'w') as f:
        f.write(string)
    print('file {} saved on disk'.format(path))


version = '5.1.0'
url = 'https://cdn.jupyter.org/notebook/{}/style/style.min.css'.format(version)
notebook_css = load_remote_file(url,
                                login='my-login',
                                password='my-password',
                                proxy_host='my-proxy.com',
                                proxy_port='8080')
notebook_css[:100]

C:\HOMEWARE\anaconda-3-x86_64\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'/*!\n*\n* Twitter Bootstrap\n*\n*/\n/*!\n * Bootstrap v3.3.7 (http://getbootstrap.com)\n * Copyright 2011-2'

## Define custom CSS
+ To hide the `Out[]` prompts
+ To slightly reduce the left margin width - when the `In[]` and `Out[]` prompts go away

In [3]:
prompt_css = """
.output_prompt {
    visibility: hidden;
}
.prompt {
    min-width: 5%; //14px
}
"""

## Define nbconvert jinja template
+ To inject the recent notebook CSS
+ To circle in RED cells (`In` and `Out`) marked as "# NBCONVERT IMPORTANT" (typically first line of code cell)
+ To remove cells (`In` and `Out`) marked as "# NBCONVERT REMOVE" (typically first line of code cell)


In [4]:
template = """
{% extends 'full.tpl'%}

{% block header %}
{{ super() }}
<style>
__notebook_css__
</style>
{% endblock header %}

{% block any_cell %}
{% if cell.get('source', '').startswith('# NBCONVERT IMPORTANT') %}
    <div style="border:thin solid red">
        {{ super() }}
    </div>
{% elif cell.get('source', '').startswith('# NBCONVERT REMOVE') %}
    <div>
    </div>
{% else %}
    {{ super() }}
{% endif %}
{% endblock any_cell %}
"""

dic = {'__notebook_css__': notebook_css}

template = multiple_replace(dic, template)

path = 'template_selective.tpl'
write_file(template, path)

file template_selective.tpl saved on disk


## Define nbconvert jinja template
+ To inject the recent notebook CSS
+ To remove all `In` cells and display only outputs
+ To hide the `Out[]` prompts and slightly reduce the left margin


In [5]:
template = """
{% extends 'full.tpl'%}

{% block header %}
{{ super() }}
<style>
__notebook_css__
__prompt_css__
</style>
{% endblock header %}

{% block input_group %}
{% endblock input_group %}
"""

dic = {'__notebook_css__': notebook_css,
       '__prompt_css__': prompt_css}

template = multiple_replace(dic, template)

path = 'template_hide_input.tpl'
write_file(template, path)

file template_hide_input.tpl saved on disk


## Choose notebook and template
+ To run nbconvert


In [6]:
nb_filename = 'sample_notebook.ipynb'

# template = 'template_selective.tpl'
template = 'template_hide_input.tpl'

print('Run nbconvert with\n\tnotebook\t{}\n\ttemplate\t{}'.format(nb_filename, template))

Run nbconvert with
	notebook	sample_notebook.ipynb
	template	template_hide_input.tpl


In [7]:
!jupyter nbconvert --to html $nb_filename --template=$template

[NbConvertApp] Converting notebook sample_notebook.ipynb to html
[NbConvertApp] Writing 577342 bytes to sample_notebook.html
